In [5]:
%cd ..
%pwd
from experiments.heap_data import *

/home/tekne/Oxford/ATML/Project


ModuleNotFoundError: No module named 'experiments'

In [3]:
wandb.init(project='ggnn_heaps')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tekne (use `wandb login --relogin` to force relogin)
/home/tekne/.local/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [4]:
wandb.finish()